In [1]:
import sys
sys.path.insert(1, '../Modules/')
from chunck import Chunck
from segmentator import Segmentator

In [2]:
seg = Segmentator('../Models/vosk-model-small-ru-0.4/', 
                 '../Models/shape_predictor_68_face_landmarks.dat', 1)

In [3]:
seg.segment_to_chuncks('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4',
                       'TestDataSet/')

chunk:   3%|▎         | 153/5319 [00:00<00:03, 1441.37it/s, now=None]

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


NoBackendError: 

In [1]:
import os
import sys
import subprocess
import json
import math

import cv2
import vosk
import librosa
import pytube as pt
import numpy as np
import pandas as pd
import moviepy.editor as mp

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

sys.path.insert(1, '../Modules/')
from chunck import Chunck

In [8]:
class Segmentator:
	"""Класс для сегментации видоефайлов"""
	def __init__(self, audio_model_path, landmarks_model_path, conf=1):
		self.landmarks_model_path = landmarks_model_path
		self.audio_model_path = audio_model_path
		self.conf = conf # Вероятность, начиная с которой надо оставлять распознанные слов
		self.word_dict = None
		

	def segment_to_chuncks(self, path, out_path):
		"""
		Метод для сегментации видеофайла по словам в датасет

		path: путь для видео
		out_path: путь для датасета
		"""
		if self.word_dict is None:
			self.word_dict = self.segment_words(path)

		for key in self.word_dict:
			for index, word in enumerate(self.word_dict[key]):
				max_index = 0

				if word[2] >= self.conf:
					l_path = f'{out_path}/{word[0]}'
					w_path = f'{out_path}/{word[0]}/{word}'

					if not os.path.exists(l_path):
						os.mkdir(l_path)
					if not os.path.exists(w_path):
						os.mkdir(w_path)
					else:

						files = os.listdir()
						for file in files:
							max_index = max(max_index, file.partition('.'))

					temp_path = f'{w_path}/{max_index}t.mp4'
					ffmpeg_extract_subclip(path, word[0], word[1], temp_path)

					ch = Chunck(path, self.landmarks_model_path)
					ch.prepare()
					ch.to_file(f'{w_path}/{max_index}.mp4')

					if os.path.isfile(temp_path):
						os.remove(temp_path)
						

	def _extract_words(self, res):
		jres = json.loads(res)
		if not 'result' in jres:
			return []
		words = jres['result']
		return words

	def _transcribe_words(self, recognizer, bytes):
		result = []

		chunck_size = 4000
		for chunck_no in range(math.ceil(len(bytes)) / chunck_size):
			start = chunck_size * chunck_no
			end = min(len(bytes), (chunck_no + 1) * chunck_size)
			data = bytes[start : end]

			if recognizer.AcceptWaveform(data):
				words = self._extract_words(recognizer.Result())
				result += words

		result += self._extract_words(recognizer.FinalResult())

		return result

	def segment_words(self, input_path, out_path=None):
		"""
		Метод для сбора информации о таймингах слов в
		словарь или cvs файл

		input_path: путь к видеофайлу
		out_path: путь к csv файлу. Если None, то сохранять не надо

		returns: словарь слов с таймингами и вероятностями
		"""

		clip = mp.VideoFileClip(input_path)
		temp_audio_path = 'audio.mp3'
		clip.audio.write_audiofile(temp_audio_path)

		vosk.SetLogLevel(-1)

		sample_rate = 16000
		audio, sr = librosa.load(temp_audio_path, sr=sample_rate)

		int16 = np.int16(audio * 32768).tobytes()

		model = vosk.Model(self.audio_model_path)
		recognizer = vosk.KaldiRecognizer(model, sample_rate)

		res = transcribe_words(recognizer, int16)
		df = pd.DataFrame.from_records(res)
		df = df.sort_values('start')

		if out_path is not None:
			df.to_cvs(out_path, index=False)


		word_dict = {}
		for index, row in df.iterrows():
			word_dict[row['word']] = []

		for index, row in df.iterrows():
			word_dict[row['word']].append((row['start'], row['end'], row['conf']))

		if os.path.isfile(temp_audio_path):
			os.delete(temp_audio_path)

		return word_dict

In [9]:
s = Segmentator('../Models/vosk-model-small-ru-0.4/',
               '../Models/shape_predictor_68_face_landmarks.dat')

In [10]:
s.segment_to_chuncks('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4', 
                     'TestDataSet/')

chunk:   3%|▎         | 153/5319 [00:00<00:03, 1471.72it/s, now=None]

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


NoBackendError: 

In [11]:
s.segment_words('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4')

chunk:   2%|▏         | 132/5319 [00:00<00:03, 1319.60it/s, now=None]

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


NoBackendError: 

In [6]:
import sys
import os
import subprocess
import json
import math

import vosk
import librosa
import numpy as np
import pandas as pd
import moviepy.editor as mp

sys.path.insert(1, '../Modules/')
from chunck import Chunck
from segmentator import Segmentator

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [7]:
class Segmentator:
    def __init__(self, audio_model_path, video_model_path,
                conf=1):
        self.audio_model_path = audio_model_path
        self.video_model_path = video_model_path
        self.conf = conf
        
    def _to_dict(self, df):
        result = {}
        for index, row in df.iterrows():
            if row['conf'] >= self.conf:
                result[row['word']] = []

        for index, row in df.iterrows():
            if row['conf'] >= self.conf:
                result[row['word']].append((row['start'], row['end']))
        return result
        
    def _extract_words(self, res):
        jres = json.loads(res)
        if not 'result' in jres:
            return []
        words = jres['result']
        return words
    
    def _transcribe_words(self, recognizer, bytes):
        result = []

        chunk_size = 4000
        for chunk_no in range(math.ceil(len(bytes) / chunk_size)):
            start = chunk_no * chunk_size
            end = min(len(bytes), (chunk_no + 1) * chunk_size)
            data = bytes[start : end]

            if recognizer.AcceptWaveform(data):
                words = self._extract_words(recognizer.Result())
                result += words
        result += self._extract_words(recognizer.FinalResult())

        return result
    
    def get_info(self, input_path, out_path=None):
        vosk.SetLogLevel(-1)
        
        temp_audio_path = 'temp.wav'
        clip = mp.VideoFileClip(input_path)
        clip.audio.write_audiofile(temp_audio_path)

        sample_rate = 16000
        audio, sr = librosa.load(temp_audio_path, sr=sample_rate)

        int16 = np.int16(audio * 32768).tobytes()

        model = vosk.Model(self.audio_model_path)
        recognizer = vosk.KaldiRecognizer(model, sample_rate)

        res = self._transcribe_words(recognizer, int16)
        df = pd.DataFrame.from_records(res)
        df = df.sort_values('start')
        
        if os.path.isfile(temp_audio_path):
            os.remove(temp_audio_path)
        
        if out_path is not None:
            df.to_csv(out_path, index=False)
            
        return self._to_dict(df)
    
    def get_chuncks(self, input_path, out_path='', size=(50, 50),
                    word_dict=None):
        # Create a dict for segmented words timestamps
        if word_dict is None:
            word_dict = self.get_info(input_path)
        
        if not os.path.exists(out_path):
            os.mkdir(out_path)
            
        for word in word_dict:
            w_num = 0
            l_path = f'{out_path}/{word[0]}'
            w_path = f'{l_path}/{word}'
            
            if not os.path.exists(l_path):
                os.mkdir(l_path)
                
            if not os.path.exists(w_path):
                os.mkdir(w_path)
              
            filenames = os.listdir(w_path)
            for filename in filenames:
                string_num = filename.partition('.')[0]
                if string_num.isdigit():
                    w_num = max(w_num, int(filename.partition('.')[0]))
                
            for current_word in word_dict[word]:
                print(current_word)
                temp_video_path = f'{w_path}/temp.mp4'

                ffmpeg_extract_subclip(input_path, current_word[0], 
                                       current_word[1],
                                       targetname=temp_video_path)
                
                ch = Chunck(temp_video_path, self.video_model_path, size)
                if not ch.prepare():
                    continue
                    
                try:
                    ch.to_file(f'{w_path}/{w_num}.avi')
                except ZeroDivisionError:
                    continue
                
                df = pd.DataFrame(columns=['length'],
                                 data=[current_word[1] - current_word[0]])
                df.to_csv(f'{w_path}/{w_num}.csv', index=False)
                
                w_num += 1
                
                if os.path.isfile(temp_video_path):
                    os.remove(temp_video_path)
                    
                    
        pass
    

In [2]:
s = Segmentator('../Models/vosk-model-small-ru-0.4/', 
                '../Models/shape_predictor_68_face_landmarks.dat')

In [41]:
d = s.get_info('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4')

chunk:   5%|▌         | 281/5319 [00:00<00:01, 2809.90it/s, now=None]

MoviePy - Writing audio in temp.wav


MoviePy - Done.


In [3]:
s.get_chuncks('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4', 
              'TestDataSet/')

chunk:   6%|▌         | 294/5319 [00:00<00:01, 2933.10it/s, now=None]

MoviePy - Writing audio in temp.wav


MoviePy - Done.
(0.904665, 1.2)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(1.2, 1.41)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(123.45, 123.75)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(207.27, 207.57)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(1.41, 1.59)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(225.78, 226.02)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(1.71, 2.43)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(226.05, 226.5)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(3.96, 4.501097)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(4.62, 5.01)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(5.01, 5.49)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(6.75, 7.23)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy 

KeyboardInterrupt: 

In [16]:
ch = Chunck('../Parsing/Audio/Dataset/з/забавно/0.mp4',
            '../Models/shape_predictor_68_face_landmarks.dat')

In [17]:
ch.prepare()

True

In [19]:
ch.show(0.5)

(19.270833333333336, 14.062499999999996)

In [5]:
print(ch.x_max_landmark, ch.x_min_landmark)

423 388


In [5]:
ch.to_file('test1.avi')

In [5]:
import sys
sys.path.insert(1, '../Modules/')
from chunck import Chunck
from segmentator import Segmentator

In [ ]:
s = Segmentator('../Models/shape_predictor_68_face_landmarks.dat',
                '../Models/vosk-model-small-ru-0.4/')


TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

In [4]:
import cv2
import time
import dlib
import numpy as np

In [19]:
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../Models/shape_predictor_68_face_landmarks.dat')
length = 60 - 48
x, prev_x = np.zeros(length), np.zeros(length)
y, prev_y = np.zeros(length), np.zeros(length)
velocity_x, velocity_y = 0, 0
detections = 0

fps = cap.get(cv2.CAP_PROP_FPS)

iteration = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face).parts()[48:60]
        if iteration <= 5:
            detections += 1
        current_velocity_x, current_velocity_y = 0, 0
        for i, landmark in enumerate(landmarks):
            prev_x[i], x[i] = x[i], landmark.x
            prev_y[i], y[i] = y[i], landmark.y
            
            if iteration <= 5:
                current_velocity_x += np.abs(x[i] - prev_x[i])
                current_velocity_y += np.abs(y[i] - prev_y[i])
            
            cv2.circle(frame, (landmark.x, landmark.y),
                      2, (0, 0, 255))
            
        current_velocity_x /= length
        current_velocity_y /= length
        velocity_x += current_velocity_x
        velocity_y += current_velocity_y
        
    print(velocity_x, velocity_y)
        
            
    cv2.imshow('video', frame)
        
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

velocity_x /= detections
velocity_y /= detections

velocity_x *= fps
velocity_y *= fps

print(velocity_x, velocity_y)

331.8333333333333 360.5
332.5 361.25
333.1666666666667 361.4166666666667
333.6666666666667 361.9166666666667
334.0 362.58333333333337
334.3333333333333 362.6666666666667
335.91666666666663 363.1666666666667
336.24999999999994 364.08333333333337
336.74999999999994 365.50000000000006
337.24999999999994 367.75000000000006
338.33333333333326 370.00000000000006
338.99999999999994 371.66666666666674
339.49999999999994 375.75000000000006
340.33333333333326 377.91666666666674
341.74999999999994 379.91666666666674
342.24999999999994 381.0833333333334
343.83333333333326 382.7500000000001
344.74999999999994 383.5833333333334
345.33333333333326 384.5000000000001
345.83333333333326 384.8333333333334
346.49999999999994 385.41666666666674
346.91666666666663 385.8333333333334
348.3333333333333 386.5833333333334
349.0833333333333 388.0833333333334
349.75 389.8333333333334
350.75 391.0000000000001
351.4166666666667 392.3333333333334
351.6666666666667 393.0833333333334
352.4166666666667 393.4166666666667

In [11]:
detections

1068

In [1]:
import pandas as pd

In [14]:
df = pd.DataFrame(columns=['l', 'v1', 'v2'],
                  data=[[2, 4, 5]])

In [6]:
df = pd.DataFrame(columns=['url'],
                 data=[['asdsfdsf'], ['sdfdsfds'],
                       ['sdfdsfdsfdsf'],
                       ['sdfdsfdsf']])

In [14]:
n = df.to_numpy()
for el in n:
    print(el[0])

asdsfdsf
sdfdsfds
sdfdsfdsfdsf
sdfdsfdsf


In [2]:
import pytube as pt

In [3]:
class Parser:
    def __init__(self, seg, report_path):
        self.seg = seg
        self.report_path = report_path
        self.columns = ['length', 'velocity_x', 'velocity_y']
        
    def parse(self, url, out_path):
        youtube = pt.YouTube(url)
        temp_video_path = 'temp.mp4'
        video = youtube.streams.first().download(filename='temp')
        report = seg.get_chuncks(temp_video_path, out_path)
        
        if os.path.exists(temp_video_path):
            os.remove(temp_video_path)
            
        return report
        
    def from_cvs(self, csv_path, out_path):
        urls_df = pd.read_csv(csv_path)
        urls_temp = urls_df.to_numpy()
        urls = []
        for url in urls_temp:
            urls.append(url[0])
        urls = np.array(urls, dtype=str)
        report = pd.DataFrame(columns=self.columns)
        
        for url in urls:
             report = pd.concat([report, self.parse(url, out_path)])
                
        report.to_csv(self.report_path)
        
        return report


In [20]:
df1 = pd.DataFrame(columns=['a', 'b'], data=[[1, 2]])
df2 = pd.DataFrame(columns=['a', 'b'], data=[[4, 5]])

In [25]:
pd.concat([df1, df2], ignore_index=True)

,a,b
0,1,2
1,4,5


In [8]:
test_df = pd.DataFrame(columns=['url'], data=[['https://www.youtube.com/watch?v=k4J9WGijj7o'],
                                    ['https://www.youtube.com/watch?v=CLuoNrkO8mg']])

In [10]:
test_df.to_csv('test.csv', index=False)

In [1]:
import sys
sys.path.insert(1, '../Modules/')
from segmentator import Segmentator
from parser import Parser

In [2]:
seg = Segmentator('../Models/vosk-model-small-ru-0.4/',
                  '../Models/shape_predictor_68_face_landmarks.dat')

In [3]:
p = Parser(seg, 'report.csv')

In [4]:
p.from_cvs('test.csv', 'TestDataSet3/')

chunk:   3%|▎         | 225/6930 [00:00<00:02, 2246.61it/s, now=None]

MoviePy - Writing audio in temp.wav


MoviePy - Done.
(1.86, 2.07)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(2.07, 2.43)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(2.46, 2.67)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(2.67, 2.88)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(2.88, 3.15)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(3.15, 3.290655)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(63.27, 63.45)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(3.78, 4.17)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(4.92, 5.28)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(5.85, 6.3)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(6.3, 6.42)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(14.79, 14.91)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command succe

(285.57, 285.72)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(21.228036, 21.48)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(21.48, 21.66)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(31.47, 31.74)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(42.57, 42.75)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(55.74, 55.921318)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(114.42, 114.6)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(132.72, 132.84)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(240.39, 240.54)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(260.49, 260.64)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(21.66, 22.08)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(22.26, 22.47)
Moviepy - Running:
>>> "+ " ".join(cmd)
Movi

(97.26, 97.38)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(119.61, 119.91)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(123.27, 123.39)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(154.8, 154.951611)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(67.17, 67.62)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(67.62, 68.04)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(68.1, 68.67)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(76.479932, 77.04)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(146.61, 147.09)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(189.24, 189.66)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(68.67, 69.0)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(147.09, 147.42)
Moviepy - Running:
>>> "+ " ".join(cmd)
Movi

...

(69.87, 70.29)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(77.58, 77.82)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(78.93, 79.29)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(79.38, 79.56)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(180.81, 181.05)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(335.82, 336.15)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(341.302939, 341.520732)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(79.98, 80.28)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(83.34, 83.7)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(83.7, 84.06)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(86.43, 86.76)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(167.85, 168.15)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy

(146.102051, 146.43)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(146.43, 147.0)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(147.06, 147.18)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(157.62, 157.92)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(226.77, 226.92)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(147.18, 147.42)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(148.83, 149.22)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(208.08, 208.44)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(329.07, 329.4)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(345.87, 346.26)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(357.18, 357.45)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(149.22, 149.725107)
Moviepy - Running:
>>> "+ " ".j

(217.02, 217.47)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(217.68, 218.07)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(218.28, 218.82)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(220.26, 220.8)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(220.8, 221.34)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(221.34, 221.76)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(221.76, 221.94)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(222.45, 222.72)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(223.56, 223.86)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(224.49, 224.67)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(226.08, 226.2)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(311.28, 311.49)
Moviepy - Running:
>>> "+ " ".join(cmd)


(306.39, 306.9)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(310.5, 311.04)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(311.49, 312.03)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(312.06, 312.48)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(314.07, 314.67)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(316.62, 317.07)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(320.37, 320.64)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(320.94, 321.33)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(322.14, 322.56)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(322.86, 323.16)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(323.61, 324.03)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
(324.03, 324.48)
Moviepy - Running:
>>> "+ " ".join(cmd)

,length,velocity_x,velocity_y
0,0.210000,14.583333,36.111111
0,0.360000,26.785714,17.857143
0,0.210000,45.833333,45.138889
0,0.210000,16.666667,14.583333
0,0.270000,12.916667,20.833333
...,...,...,...
0,0.241846,33.333333,31.770833
0,0.630000,17.708333,25.892857
0,0.300000,50.347222,31.597222
0,0.330000,32.638889,18.055556
